In [16]:
get_ipython().system_raw("mlflow ui --port 5090 &")

In [17]:
from pyngrok import ngrok

In [36]:
ngrok.kill()

In [19]:
!ngrok config add-authtoken 2sVwQYmf9x00hLx9OsT04T4Vk1R_3gyuN842eLrdSZYgebt5F

Authtoken saved to configuration file: /home/workbench/.config/ngrok/ngrok.yml


[2025-02-04 06:06:17 +0000] [2495] [ERROR] Connection in use: ('127.0.0.1', 5090)
[2025-02-04 06:06:17 +0000] [2495] [ERROR] connection to ('127.0.0.1', 5090) failed: [Errno 98] Address already in use


In [20]:
ngrok_tunnel = ngrok.connect(addr="5090", proto="http", bind_tls=True)

[2025-02-04 06:06:18 +0000] [2495] [ERROR] Connection in use: ('127.0.0.1', 5090)
[2025-02-04 06:06:18 +0000] [2495] [ERROR] connection to ('127.0.0.1', 5090) failed: [Errno 98] Address already in use
[2025-02-04 06:06:19 +0000] [2495] [ERROR] Can't connect to ('127.0.0.1', 5090)
Running the mlflow server failed. Please see the logs above for details.


In [21]:
print("MLflow UI ", ngrok_tunnel.public_url)

MLflow UI  https://a1e3-67-254-220-141.ngrok-free.app


In [22]:
import random
import string
import yaml
import requests
import json


import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.pytorch

In [23]:
def generate_random_experiment_name():
    """Generate a random experiment name."""
    prefix = "experiment"
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    return f"{prefix}-{random_suffix}"

In [34]:
mlflow.pytorch.autolog()

#mlflow set experiement
mlflow.tracking.set_tracking_uri("https://a1e3-67-254-220-141.ngrok-free.app/")
experiment_name=generate_random_experiment_name()
print(experiment_name)
mlflow.set_experiment(experiment_name=experiment_name)

with mlflow.start_run(run_name = "new-pytorch-run") as run:
    dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')
    
    dataset.describe()
    
    dataset.head()
    
    X = dataset.iloc[:, :-1].values
    y = dataset.iloc[:,-1].values
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)
    
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    Xtrain_ = torch.from_numpy(X_train).float()
    Xtest_ = torch.from_numpy(X_test).float()
    
    Xtrain_
    
    ytrain_ = torch.from_numpy(y_train)
    ytest_ = torch.from_numpy(y_test)
    
    ytrain_
    
    Xtrain_.shape, ytrain_.shape
    
    Xtest_.shape, ytest_.shape
    
    input_size=2
    output_size=2
    hidden_size=10
    
    class Net(nn.Module):
       def __init__(self):
           super(Net, self).__init__()
           self.fc1 = torch.nn.Linear(input_size, hidden_size)
           self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
           self.fc3 = torch.nn.Linear(hidden_size, output_size)
    
    
       def forward(self, X):
           X = torch.relu((self.fc1(X)))
           X = torch.relu((self.fc2(X)))
           X = self.fc3(X)
    
           return F.log_softmax(X,dim=1)
    
    model = Net()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.NLLLoss()
    
    epochs = 50
    
    for epoch in range(epochs):
      optimizer.zero_grad()
      Ypred = model(Xtrain_)
      loss = loss_fn(Ypred,  ytrain_)
      loss.backward()
      optimizer.step()
      print('Epoch',epoch, 'loss',loss.item())
    
    mlflow.end_run()

    # mlflow.log_param("epochs", 50)
    # mlflow.pytorch.log_model(model, "models")
    # mlflow.log_metric("loss", loss.item())

2025/02/04 07:34:02 WARNING mlflow.utils.autologging_utils: MLflow pytorch autologging is known to be compatible with 1.9.0 <= torch <= 2.5.1, but the installed version is 2.1.0a0+32f93b1. If you encounter errors during autologging, try upgrading / downgrading torch to a compatible version, or try upgrading MLflow.
2025/02/04 07:34:02 INFO mlflow.tracking.fluent: Experiment with name 'experiment-cw5uk0zo' does not exist. Creating a new experiment.


experiment-cw5uk0zo
Epoch 0 loss 0.7829716801643372
Epoch 1 loss 0.7530030608177185
Epoch 2 loss 0.7241042852401733
Epoch 3 loss 0.6963628530502319
Epoch 4 loss 0.6693798303604126
Epoch 5 loss 0.6434211730957031
Epoch 6 loss 0.6186364889144897
Epoch 7 loss 0.5952240228652954
Epoch 8 loss 0.5733435750007629
Epoch 9 loss 0.5530135631561279
Epoch 10 loss 0.534273624420166
Epoch 11 loss 0.5174434185028076
Epoch 12 loss 0.5023916363716125
Epoch 13 loss 0.4887007772922516
Epoch 14 loss 0.4763617217540741
Epoch 15 loss 0.4649014472961426
Epoch 16 loss 0.4543777108192444
Epoch 17 loss 0.4442989230155945
Epoch 18 loss 0.4347800314426422
Epoch 19 loss 0.4251977503299713
Epoch 20 loss 0.41589486598968506
Epoch 21 loss 0.40656712651252747
Epoch 22 loss 0.3981384336948395
Epoch 23 loss 0.3899211585521698
Epoch 24 loss 0.38207682967185974
Epoch 25 loss 0.373961865901947
Epoch 26 loss 0.36602187156677246
Epoch 27 loss 0.3582989275455475
Epoch 28 loss 0.350745290517807
Epoch 29 loss 0.3433869481086731

In [12]:
ngrok.kill()